In [101]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import random
import sqlite3
import pandas as pd

In [124]:
url = 'https://www.perekrestok.ru/cat/d?page='

In [86]:
# Функция, которая парсит бесплатные прокси.
# На вход получает html
# Возвращает список с прокси. Протокол выбран только http
def get_proxies(html):
    final_proxies_list = []
    soup = BeautifulSoup(html.content, 'html.parser')
    proxies_list = soup.find('tbody').find_all('tr')
    for j in proxies_list:
        proxy_row = j.find_all('td')
        ip = proxy_row[0].text
        port = proxy_row[1].text
        if proxy_row[6].text == 'no':
            protocol = 'http'
            proxy = protocol + '://' + ip + ':' + port
            final_proxies_list.append(proxy)
        else:
            continue
    return final_proxies_list
proxies_list = get_proxies(requests.get('https://free-proxy-list.net/'))

['http://47.88.62.42:80',
 'http://134.195.101.34:8080',
 'http://4.175.121.88:80',
 'http://139.59.1.14:8080',
 'http://12.186.205.123:80',
 'http://162.223.94.164:80',
 'http://137.184.100.135:80',
 'http://190.103.177.131:80',
 'http://161.35.70.249:3128',
 'http://188.166.56.246:80',
 'http://51.15.242.202:8888',
 'http://178.21.163.24:80',
 'http://185.140.53.137:80',
 'http://116.203.28.43:80',
 'http://198.98.60.66:80',
 'http://194.67.91.153:80',
 'http://118.69.134.2:80',
 'http://164.163.185.204:80',
 'http://95.216.164.36:80',
 'http://194.163.132.232:3128',
 'http://114.156.77.107:8080',
 'http://43.157.67.116:8888',
 'http://72.10.160.90:18821',
 'http://64.225.4.17:10005',
 'http://117.54.114.100:80',
 'http://103.125.160.178:80',
 'http://67.43.236.20:27293',
 'http://113.161.131.43:80',
 'http://103.147.118.237:8080',
 'http://47.56.110.204:8989',
 'http://77.94.137.243:3128',
 'http://162.223.94.163:80',
 'http://128.199.202.122:8080',
 'http://75.89.101.62:80',
 'http

In [201]:
prod_list,price_list,old_price_list=[],[],[]
url = 'https://www.perekrestok.ru/cat/d?page='

for i in range(1,30):
    r = requests.get(url+str(i), {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
    soup = BeautifulSoup(r.content, 'html.parser')
    for i in soup.find_all(class_='product-card__link-text'):
        prod_list.append(str(i).split('<span class="product-card__link-text">')[1].replace('</span>',''))

    for i in soup.find_all(class_='product-card__price'):
        new = i.find_all(class_='price-new')
        old = i.find_all(class_='price-old')
        price_list.append(float(str(*new).replace('<div class="price-new">','').replace('₽</div>','').replace(',','.').replace(' ','').strip()))
        if str(*old).replace('<div class="price-old">','').replace('₽</div>','').replace(',','.').replace(' ','').strip()=="":
            old_price_list.append(0)
        else:
            old_price_list.append(float(str(*old).replace('<div class="price-old">','').replace('₽</div>','').replace(',','.').replace(' ','').strip()))

In [204]:
df = pd.DataFrame({'prod':prod_list, 'price':price_list,'old_price':old_price_list})
df

,prod,price,old_price
0,"Йогурт Teos Греческий 2%, 140г",37.99,45.99
1,Яйцо куриное активита С1 с селеном и йодом Зел...,92.99,104.99
2,Апельсины,169.99,199.99
3,Сырок творожный Ростагроэкспорт с варёной сгущ...,33.74,0.00
4,Корм Felix Аппетитные кусочки с курицей в желе...,19.99,31.99
...,...,...,...
1387,"Гель для душа Освежающий 3в1 Маркет, 790мл",219.99,259.99
1388,"Пюре Semper овощи с фрикадельками из индейки, ...",199.99,249.99
1389,"Миндаль жареный Маркет Collection, 135г",219.99,259.99
1390,"Средство Лактацид для интимной Гигиены, 200мл",369.99,459.00


In [205]:
df['inf'] = df['prod'].str.split(',').str[0]
df['char'] = df['prod'].str.split(',').str[1]
df = df.drop('prod',axis=1)
df.loc[df.old_price==0,'old_price'] = df.loc[df.old_price==0,'price']

In [225]:
df['discount'] = (df['old_price'] - df['price'])/df['old_price']

In [236]:
df.sort_values(by = 'discount',ascending = False).head(40)

,price,old_price,inf,char,discount
1134,9.99,34.99,Тетрадь Пироженка ученическая в линию 017506,12л,0.714490
1295,39.99,95.99,Щербет Азовская КФ с какао и арахисом,200г,0.583394
1033,199.99,459.99,Торт Тирамису,450г,0.565230
342,169.99,379.99,Ветчина варёная Мясной Дом Бородина Для завтрака,330г,0.552646
643,59.99,129.99,Шоколад молочный Россия-Щедрая Душа с карамель...,82г,0.538503
131,149.99,319.99,Колбаса Вязанка Молокуша вареная,450г,0.531267
709,154.99,329.99,Конфеты Красный Октябрь Мишка косолапый,200г,0.530319
667,79.99,169.99,Крабовые палочки Vici имитация охлаждённые,220г,0.529443
627,79.99,169.99,Крабовое мясо Vici имитация охлаждённое,220г,0.529443
500,46.99,97.99,Макароны Makfa длинные,500г,0.520461


In [61]:
import pandas as pd
df=pd.DataFrame(perek_milk_price, columns = ['название','цена'])
df.to_excel('./milk.xlsx',sheet_name='perek_milk' , index_label='id' )
